In [5]:
library("btergm")
library("dplyr")
library("Matrix")
library("statnet")  
library("texreg") 
library("data.table")
library("metafor")

In [6]:
#Data Preparation 

#Generate edge matrix and adjust for missing players
create_round_edges <- function(data, players){
  players_in_game <- union(unique(data$player_id), unique(data$other_id))
  edge_mat <- matrix(0,nrow=players,ncol=players)
  data <- adjust_missing_players(data, players_in_game)
  for(j in 1:nrow(data)){
    player_id <- data[[j, "player_id"]]
    other_id <- data[[j, "other_id"]]
    edge_mat[player_id, other_id] <- 1
  }
  return(edge_mat)
}

#Add Endowment as a vertex attribute to the network
add_endowment <- function(network, data){
  endowment <- unique(data[(data$round==1), ])
  endowment <- endowment[ , c("player_id", "endowment")][order(endowment$player_id), ]
  network <- set.vertex.attribute(network, "endowment", endowment[,2])
  return(network)
}

#Adjust wealth for missing players and add it to the network
add_wealth <- function(network, data, round){
  wealth_data <- as.data.table(data)
  wealth_data <- wealth_data[ , wealth := shift(final_wealth, fill=0), by=c('treatment', 'group_id', 'player_id')]
  wealth_data <- transform(wealth_data, wealth=sqrt(wealth))
  players_in_game <- wealth_data[(wealth_data$round==1), ]$player_id
  wealth_data <- adjust_missing_players(wealth_data, players_in_game)
  setDF(wealth_data)
  round_wealth <- get_round_wealth(wealth_data, round)
  network <- set.vertex.attribute(network, "wealth", round_wealth)
  return(network)
}

#Adjust ID's to account for missing players
adjust_missing_players <- function(data, players_in_game){
  num_players <- length(players_in_game)
  if (num_players < max(players_in_game)) {
    x <- setdiff(seq(1, max(players_in_game)), players_in_game) 
    data$player_id[data$player_id > x] <- data$player_id[data$player_id > x] - 1
    if("other_id" %in% colnames(data))
    {
      data$other_id[data$other_id > x] <- data$other_id[data$other_id > x] - 1
    }
  }
  return(data)
}

#Returns the wealth of a player for in a specific round
get_round_wealth <- function(data, current_round){
  round_data <- data[(data$round==current_round), ]
  return(round_data[,"wealth"])
}

#Generate the network in the correct format and add the outdegree as a vertex atrribute
get_btergm_data <- function(data1, data2){
  network <- list()
  players_in_game <- union(unique(data1$player_id), unique(data1$other_id))
  num_players <- length(players_in_game)
  for(i in 1:20){
    round_edges <- create_round_edges(data1[(data1$round == i), ], num_players)
    network[[i]] <- network(round_edges)
    network[[i]] <- add_endowment(network[[i]], data2)
    network[[i]] <- add_wealth(network[[i]], data2, i)
    outdegree <- degree(network[[i]], cmode = "outdegree")
    network[[i]] <- set.vertex.attribute(network[[i]], "outdegree", outdegree)
  }
  return(network)
}

#   Metafor package

#Extract model data for metafor
get_metafor_data <- function(group_models){
  terms <- btergm.se(group_models[[1]])
  yis <-matrix(list(), nrow=nrow(terms), ncol=length(group_models))
  seis <- matrix(list(), nrow=nrow(terms), ncol=length(group_models))
  for(i in 1:length(group_models)){
    model <- group_models[[i]]
    stats <- btergm.se(model)
    for(j in 1:nrow(stats)){
      yis[i,j] = stats[j,1] #Gets the coef estimate
      seis[i,j] = stats[j,2] #Gets the coef standart error
    }
  }
  return(export_coefs(btergm08(yis,seis,terms),terms))
}


#Run meta-analysis using all group models for each coefficient
btergm08 <- function(yis,seis,terms){
  yis <- split(yis, rep(1:ncol(yis), each = nrow(yis)))
  seis <- split(seis, rep(1:ncol(seis), each = nrow(seis)))
  coefs <- list()
  for(i in 1:nrow(terms)){
    yi <- yis[i]
    sei <- seis[i]
    yi<- unlist(yi, use.names=FALSE)
    sei<- unlist(sei, use.names=FALSE)
    coef <- rownames(terms)[i]
    new.term <- data.frame(yi, sei)
    meta<- rma.uni(yi, sei, data = new.term, method = "REML", level = 95)
    coefs[[coef]] <- meta
  }
  return (coefs)
}

#Create coefficient matrix
export_coefs <- function(coefs,terms){
  mat <- matrix(list(), nrow = nrow(terms), ncol = 5)
  rownames(mat) <- rownames(terms)
  for(name in rownames(terms)){
    term <- coefs[[name]]
    est <- as.numeric(term["beta"])
    se <- as.numeric(term["se"])
    lb <-as.numeric(term["ci.lb"])
    ub <-as.numeric(term["ci.ub"])
    lev <- 1-as.numeric(term["level"])
    mat[name,] <- c(est,se,lb,ub,lev)
  }
  colnames(mat) <- c("estimate","se","mu-min","mu-plus","alpha-mu")
  return(mat)
}

get_pooled_model <- function(group_models){
  terms <- btergm.se(group_models[[1]])
  yis <-matrix(list(), nrow=length(group_models), ncol=nrow(terms))
  seis <- matrix(list(), nrow=length(group_models), ncol=nrow(terms))
  numobs <- c()
  print(nrow(terms))
  print(length(group_models))
  for(i in 1:length(group_models)){
    model <- group_models[[i]]
    stats <- btergm.se(model)
    numobs[i] <- as.numeric(nobs(model)[2])
    for(j in 1:nrow(stats)){
      yis[i,j] = stats[j,1] #Gets the coef estimate
      seis[i,j] = stats[j,2] #Gets the coef standart error
    }
  }
  btergm082(yis,seis,numobs,terms)
}

btergm082 <- function(yis,seis,numobs,terms){
  yis <- split(yis, rep(1:ncol(yis), each = nrow(yis)))
  seis <- split(seis, rep(1:ncol(seis), each = nrow(seis)))
  pooled_effs <- list()
  pooled_se <- list()
  for(i in 1:nrow(terms)){
    yi <- yis[i]
    sei <- seis[i]
    n <- numobs[i]
    yi<- unlist(yi, use.names=FALSE)
    sei<- unlist(sei, use.names=FALSE)
    pooled_effs[i] <- mean(yi)
    pooled_se[i] <- get_pooled_SE(sei,numobs)
  }
  return(export_coefs2(pooled_effs,pooled_se, terms))
}

export_coefs2 <- function(pooled_effs,pooled_se,terms){
  mat <- matrix(list(), nrow = nrow(terms), ncol = 5)
  rownames(mat) <- rownames(terms)
  for(i in 1:nrow(terms)){
    est <- as.numeric(pooled_effs[i])
    se <- as.numeric(pooled_se[i])
    lb <-est-1.96*se
    ub <-est+1.96*se
    lev <- 0.95
    mat[i,] <- c(est,se,lb,ub,lev)
  }
  colnames(mat) <- c("estimate","se","mu-min","mu-plus","alpha-mu")
  return(mat)
}


#Helper SD func TODO make it SE
get_pooled_SE <- function(sei,numobs){
  numer <- 0
  denom <-0
  for(i in 1:length(sei)){
    n <- numobs[i]
    sd <- sei[i]*sqrt(numobs[i])
    sdpwd <-(sd^2)
    numer <- numer + (n-1)*sdpwd
    denom <- denom + n - 1
  }
  return(sqrt(numer/denom)/sqrt((denom + length(sei))))
}

#Test function
summarize_models <- function(models1, models2, models3){
  summary_model1 <- get_pooled_model(models1)
  summary_model2 <- get_pooled_model(models2)
  summary_model3 <- get_pooled_model(models3)
  write.csv(summary_model1, paste('btergm_version2_params_', '.csv', sep=''))
  write.csv(summary_model2, paste('btergm_no_edges_params_', '.csv', sep=''))
  write.csv(summary_model3, paste('btergm_no_edges_nodematch_params_', '.csv', sep=''))
  #Test
  print(summary_model1)
  print(summary_model2)
  print(summary_model3)
  #make for loop
  test <- merge(summary_model1,summary_model2,by="row.names",all.x=TRUE)
  test <- test[, 2:11]
  rownames(test) <- rownames(summary_model1)
  test <- merge(test,summary_model3,by="row.names",all.x=TRUE)
  return(test)
}

export_model <- function(model, model_name){
    summary <- get_pooled_model(model)
    df <- data.frame(matrix(unlist(summary), ncol=length(summary)))
    write.csv(summary, paste('btergm_output/', model_name, '.csv', sep=''))
    
}





Load Data #TODO

In [7]:
interact_data <- read.csv("data/interactions.csv", header=T)
wealth_data <- read.csv("data/finwealth_from_interactions.csv", header=T)

Run Models #TODO

In [10]:
all_networks <- list()
all_models <- list()
for (t in unique(interact_data$treatment)) {
  group_networks <- list()
  group_models1 <- list()
  group_models2 <- list()
  group_models3 <- list()
  test <- list()
  for (g in unique(interact_data[(interact_data$treatment==t), ]$group_id)) {
    filtered_data1 <- interact_data[(interact_data$treatment==t) & (interact_data$group_id==g), ]
    filtered_data2 <- wealth_data[(wealth_data$treatment==t) & (wealth_data$group_id==g), ]
    btergm_network <- get_btergm_data(filtered_data1, filtered_data2)
    btergm_model1 <- btergm(btergm_network ~ edges + mutual + ttriple +
                          + nodecov("outdegree") +
                          nodeicov("endowment") + nodeocov("endowment") + nodematch("endowment")
                        + nodeicov("wealth") + nodeocov("wealth") + nodematch("wealth"), R = 100)
    btergm_model2 <- btergm(btergm_network ~ mutual + ttriple +
                             + nodecov("outdegree") +
                             nodeicov("endowment") + nodeocov("endowment") + nodematch("endowment")
                           + nodeicov("wealth") + nodeocov("wealth") + nodematch("wealth"), R = 100)
    btergm_model3 <- btergm(btergm_network ~ mutual + ttriple +
                             + nodecov("outdegree") + nodeicov("endowment") + nodeocov("endowment") 
                             + nodeicov("wealth") + nodeocov("wealth"), R = 100)
    group_networks[[paste(t, '_', g, sep="")]] <- btergm_network
    group_models1[[paste(t, '_', g, sep="")]] <- btergm_model1
    group_models2[[paste(t, '_', g, sep="")]] <- btergm_model2
    group_models3[[paste(t, '_', g, sep="")]] <- btergm_model3

  }
  #all_models[[paste(t, sep="")]] <- group_models
  #Export metafor coefficients
  #models_summary <- summarize_models(group_models1, group_models2, group_models3)
 #df <- data.frame(matrix(unlist(models_summary), ncol=length(models_summary)))
  #write.csv(models_summary, paste('btergm_output/models_params_', t, '.csv', sep=''))
  export_model(group_models1, paste('btergm_full_',t, sep=""))  
  export_model(group_models2, paste('btergm_no_edges_',t, sep=""))
  export_model(group_models3, paste('btergm_no_edges_nodematch_',t, sep=""))  
  all_networks[[paste(t, sep="")]] <- group_networks
}





No covariates provided.

All networks are conformable.

Starting pseudolikelihood estimation with 100 bootstrapping replications on a single computing core...
Done.

No covariates provided.

All networks are conformable.

Starting pseudolikelihood estimation with 100 bootstrapping replications on a single computing core...
Done.

No covariates provided.

All networks are conformable.

Starting pseudolikelihood estimation with 100 bootstrapping replications on a single computing core...
Done.

No covariates provided.

All networks are conformable.

Starting pseudolikelihood estimation with 100 bootstrapping replications on a single computing core...
Done.

No covariates provided.

All networks are conformable.

Starting pseudolikelihood estimation with 100 bootstrapping replications on a single computing core...
Done.

No covariates provided.

All networks are conformable.

Starting pseudolikelihood estimation with 100 bootstrapping replications on a single computing core...
Done.

No 

[1] 10
[1] 10


Warning message in btergm.se(model):
“Standard errors and p values may be misleading because the distribution of bootstrapped thetas may not be normal. Please rely on the confidence intervals instead or make sure the thetas are normally distributed (e.g., using qqnorm(object@boot$t[, 1]) etc.”Warning message in btergm.se(model):
“Standard errors and p values may be misleading because the distribution of bootstrapped thetas may not be normal. Please rely on the confidence intervals instead or make sure the thetas are normally distributed (e.g., using qqnorm(object@boot$t[, 1]) etc.”Warning message in btergm.se(model):
“Standard errors and p values may be misleading because the distribution of bootstrapped thetas may not be normal. Please rely on the confidence intervals instead or make sure the thetas are normally distributed (e.g., using qqnorm(object@boot$t[, 1]) etc.”Warning message in btergm.se(model):
“Standard errors and p values may be misleading because the distribution of boots

[1] 9
[1] 10


Warning message in btergm.se(model):
“Standard errors and p values may be misleading because the distribution of bootstrapped thetas may not be normal. Please rely on the confidence intervals instead or make sure the thetas are normally distributed (e.g., using qqnorm(object@boot$t[, 1]) etc.”Warning message in btergm.se(model):
“Standard errors and p values may be misleading because the distribution of bootstrapped thetas may not be normal. Please rely on the confidence intervals instead or make sure the thetas are normally distributed (e.g., using qqnorm(object@boot$t[, 1]) etc.”Warning message in btergm.se(model):
“Standard errors and p values may be misleading because the distribution of bootstrapped thetas may not be normal. Please rely on the confidence intervals instead or make sure the thetas are normally distributed (e.g., using qqnorm(object@boot$t[, 1]) etc.”Warning message in btergm.se(model):
“Standard errors and p values may be misleading because the distribution of boots

...

Warning message in btergm.se(model):
“Standard errors and p values may be misleading because the distribution of bootstrapped thetas may not be normal. Please rely on the confidence intervals instead or make sure the thetas are normally distributed (e.g., using qqnorm(object@boot$t[, 1]) etc.”Warning message in btergm.se(model):
“Standard errors and p values may be misleading because the distribution of bootstrapped thetas may not be normal. Please rely on the confidence intervals instead or make sure the thetas are normally distributed (e.g., using qqnorm(object@boot$t[, 1]) etc.”Warning message in btergm.se(model):
“Standard errors and p values may be misleading because the distribution of bootstrapped thetas may not be normal. Please rely on the confidence intervals instead or make sure the thetas are normally distributed (e.g., using qqnorm(object@boot$t[, 1]) etc.”Warning message in btergm.se(model):
“Standard errors and p values may be misleading because the distribution of boots

[1] 9
[1] 10


Warning message in btergm.se(model):
“Standard errors and p values may be misleading because the distribution of bootstrapped thetas may not be normal. Please rely on the confidence intervals instead or make sure the thetas are normally distributed (e.g., using qqnorm(object@boot$t[, 1]) etc.”Warning message in btergm.se(model):
“Standard errors and p values may be misleading because the distribution of bootstrapped thetas may not be normal. Please rely on the confidence intervals instead or make sure the thetas are normally distributed (e.g., using qqnorm(object@boot$t[, 1]) etc.”Warning message in btergm.se(model):
“Standard errors and p values may be misleading because the distribution of bootstrapped thetas may not be normal. Please rely on the confidence intervals instead or make sure the thetas are normally distributed (e.g., using qqnorm(object@boot$t[, 1]) etc.”Warning message in btergm.se(model):
“Standard errors and p values may be misleading because the distribution of boots

[1] 7
[1] 10


Warning message in btergm.se(model):
“Standard errors and p values may be misleading because the distribution of bootstrapped thetas may not be normal. Please rely on the confidence intervals instead or make sure the thetas are normally distributed (e.g., using qqnorm(object@boot$t[, 1]) etc.”Warning message in btergm.se(model):
“Standard errors and p values may be misleading because the distribution of bootstrapped thetas may not be normal. Please rely on the confidence intervals instead or make sure the thetas are normally distributed (e.g., using qqnorm(object@boot$t[, 1]) etc.”Warning message in btergm.se(model):
“Standard errors and p values may be misleading because the distribution of bootstrapped thetas may not be normal. Please rely on the confidence intervals instead or make sure the thetas are normally distributed (e.g., using qqnorm(object@boot$t[, 1]) etc.”Warning message in btergm.se(model):
“Standard errors and p values may be misleading because the distribution of boots

In [12]:
typeof(models_summary)

[1] "list"

In [19]:
write.csv(df, paste('btergm_output/models_params_', t, '.csv', sep=''))